In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.applications import Xception
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.xception import preprocess_input

from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Input
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential, Model, load_model
from keras.callbacks import ModelCheckpoint
import zipfile
import seaborn as sns
from os import listdir, makedirs

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

this notebook, I follow the code that we take in our Lab with eng.Amiraa

In [3]:
trainData = pd.read_csv("../input/mi2023-practice/train.csv")
trainData.head()

,Image,Id
0,00022e1a.jpg,w_e15442c
1,000466c4.jpg,w_1287fbc
2,00087b01.jpg,w_da2efe0
3,001296d5.jpg,w_19e5482
4,0014cfdf.jpg,w_f22f3e3


In [4]:
ID = trainData['Id']
images_name = trainData["Image"]
classes = pd.get_dummies(ID)
df = pd.concat([images_name,classes], axis=1)
df.head(1)

,Image,new_whale,w_0013924,w_001ebbc,w_002222a,w_002b682,w_002dc11,w_0087fdd,w_008c602,w_009dc00,...,w_ff70408,w_ff7630a,w_ff94ad6,w_ffa7427,w_ffa78a5,w_ffb4e3d,w_ffbd74c,w_ffcd98e,w_ffda8b2,w_ffdab7a
0,00022e1a.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
datagen = ImageDataGenerator(rescale=1./255.,
                             horizontal_flip=True,
                             width_shift_range = 0.2,
                             height_shift_range = 0.2,
                             validation_split=0.2) #augmentation

In [6]:
image_size = 299
train_dir = '../input/mi2023-practice/train/train/'
train_generator=datagen.flow_from_dataframe(
                        dataframe=trainData,
                        directory=train_dir,
                        x_col="Image",
                        y_col="Id",
                        subset="training",
                        batch_size=8,
                        seed=50,
                        shuffle=True,
                        class_mode="categorical",
                        target_size=(image_size, image_size))

Found 7880 validated image filenames belonging to 4251 classes.


In [7]:
train_dir = '../input/mi2023-practice/train/train/'
valid_generator=datagen.flow_from_dataframe(
                        dataframe=trainData,
                        directory=train_dir,
                        x_col="Image",
                        y_col="Id",
                        subset="validation",
                        batch_size=8,
                        seed=42,
                        shuffle=True,
                        class_mode="categorical",
                        target_size=(image_size, image_size))

Found 1970 validated image filenames belonging to 4251 classes.


In [8]:
testdf = pd.read_csv('../input/mi2023-practice/sample_submission.csv', dtype=str)

In [9]:
test_datagen=ImageDataGenerator(rescale=1./255.)
test_dir = '../input/mi2023-practice/test/test/'
test_generator=test_datagen.flow_from_dataframe(
                            dataframe=testdf,
                            directory=test_dir,
                            x_col="Image",
                            y_col=None,
                            batch_size=1,
                            seed=42,
                            shuffle=False,
                            class_mode=None,
                            target_size=(image_size, image_size))

Found 15610 validated image filenames.


In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.applications import Xception
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.xception import preprocess_input

from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Input
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential, Model, load_model
from keras.callbacks import ModelCheckpoint
import zipfile
import seaborn as sns
from os import listdir, makedirs

In [11]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import matplotlib.image as mplimg
from matplotlib.pyplot import imshow

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from keras import layers
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from keras.models import Model

import keras.backend as K
from keras.models import Sequential

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [11]:
model = Sequential([
               tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(image_size, image_size, 3)),
                    layers.Conv2D(16, 3, padding='same', activation='relu'), # convolution 2D
                    layers.MaxPooling2D(), # extracting important features
                    layers.Conv2D(32, 3, padding='same', activation='relu'),
                    layers.MaxPooling2D(),
                    layers.Conv2D(64, 3, padding='same', activation='relu'),
                    layers.MaxPooling2D(),
                    layers.Flatten(),
                    layers.Dense(4300, activation='relu'),
                    layers.Dense(4251)]) # layers of one hot encoding 
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 299, 299, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 299, 299, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 149, 149, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 149, 149, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 74, 74, 64)        1

In [12]:
STEP_SIZE_TRAIN=train_generator.n//8
STEP_SIZE_VALID=valid_generator.n//8
print(STEP_SIZE_TRAIN)
print(STEP_SIZE_VALID)

985
246


In [13]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics= ['accuracy'])

In [14]:
STEP_SIZE_TRAIN=train_generator.n//8
STEP_SIZE_VALID=valid_generator.n//8
print(STEP_SIZE_TRAIN)
print(STEP_SIZE_VALID)

985
246


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

filepath="weights.h5"
checkpoint = ModelCheckpoint(filepath, verbose=1, save_best_only=True)
callbacks_list = [checkpoint, es]

history = model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=5, verbose=1,
                  shuffle = True, callbacks=callbacks_list)
gc.collect()

/tmp/ipykernel_32/2706294702.py:15: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(generator=train_generator,


Epoch 1/5
763/985 [======================>.......] - ETA: 19:05 - loss: 10.9268 - accuracy: 6.5531e-04